<a href="https://colab.research.google.com/github/ErfanulHoque/Medi_Bot/blob/main/Medi_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from abc import ABCMeta, abstractmethod
import random
import json
import pickle
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

class MediDoc(metaclass=ABCMeta):
    
    @abstractmethod
    def train_model(self):
        """ child class """

    @abstractmethod
    def request_tag(self, message):
        """ child class """

    @abstractmethod
    def get_tag_by_id(self, id):
        """ child class """

    @abstractmethod
    def request_method(self, message):
        """ child class """

    @abstractmethod
    def request(self, message):
        """ child class """
        
class MediDocBot(MediDoc):

    def __init__(self, intents, intent_methods={}, model_name="medi_bot_assis"):
        self.intents = intents
        self.intent_methods = intent_methods
        self.model_name = model_name

        if intents.endswith(".json"):
            self.load_json_intents(intents)

        self.lemmatizer = WordNetLemmatizer()

    def load_json_intents(self, intents):
        self.intents = json.loads(open(intents).read())

    def train_model(self):

        self.words = []
        self.classes = []
        documents = []
        ignore_letters = ['!', '?', ',', '.']

        for intent in self.intents['intents']:
            for pattern in intent['patterns']:
                word = nltk.word_tokenize(pattern)
                self.words.extend(word)
                documents.append((word, intent['tag']))
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])

        self.words = [self.lemmatizer.lemmatize(w.lower()) for w in self.words if w not in ignore_letters]
        self.words = sorted(list(set(self.words)))

        self.classes = sorted(list(set(self.classes)))



        training = []
        output_empty = [0] * len(self.classes)

        for doc in documents:
            bag = []
            word_patterns = doc[0]
            word_patterns = [self.lemmatizer.lemmatize(word.lower()) for word in word_patterns]
            for word in self.words:
                bag.append(1) if word in word_patterns else bag.append(0)

            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1
            training.append([bag, output_row])

        random.shuffle(training)
        training = np.array(training)

        train_x = list(training[:, 0])
        train_y = list(training[:, 1])

        self.model = Sequential()
        self.model.add(Dense(1000, input_shape=(len(train_x[0]),), activation='relu'))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(500, activation='relu'))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(len(train_y[0]), activation='softmax'))

        sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
        self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        self.hist = self.model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)

    def save_model(self, model_name=None):
        if model_name is None:
            self.model.save(f"{self.model_name}.h5", self.hist)
            pickle.dump(self.words, open(f'{self.model_name}_words.pkl', 'wb'))
            pickle.dump(self.classes, open(f'{self.model_name}_classes.pkl', 'wb'))
        else:
            self.model.save(f"{model_name}.h5", self.hist)
            pickle.dump(self.words, open(f'{model_name}_words.pkl', 'wb'))
            pickle.dump(self.classes, open(f'{model_name}_classes.pkl', 'wb'))

    def load_model(self, model_name=None):
        if model_name is None:
            self.words = pickle.load(open(f'{self.model_name}_words.pkl', 'rb'))
            self.classes = pickle.load(open(f'{self.model_name}_classes.pkl', 'rb'))
            self.model = load_model(f'{self.model_name}.h5')
        else:
            self.words = pickle.load(open(f'{model_name}_words.pkl', 'rb'))
            self.classes = pickle.load(open(f'{model_name}_classes.pkl', 'rb'))
            self.model = load_model(f'{model_name}.h5')

    def _clean_up_sentence(self, sentence):
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = [self.lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words

    def _bag_of_words(self, sentence, words):
        sentence_words = self._clean_up_sentence(sentence)
        bag = [0] * len(words)
        for s in sentence_words:
            for i, word in enumerate(words):
                if word == s:
                    bag[i] = 1
        return np.array(bag)

    def _predict_class(self, sentence):
        p = self._bag_of_words(sentence, self.words)
        res = self.model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.1
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({'intent': self.classes[r[0]], 'probability': str(r[1])})
        return return_list

    def _get_response(self, ints, intents_json):
        try:
            tag = ints[0]['intent']
            list_of_intents = intents_json['intents']
            for i in list_of_intents:
                if i['tag']  == tag:
                    result = random.choice(i['responses'])
                    break
        except IndexError:
            result = "I apologize, I don't understand!"
        return result

    def request_tag(self, message):
        pass

    def get_tag_by_id(self, id):
        pass

    def request_method(self, message):
        pass

    def request(self, message):
        ints = self._predict_class(message)

        if ints[0]['intent'] in self.intent_methods.keys():
            self.intent_methods[ints[0]['intent']]()
        else:
            print(self._get_response(ints, self.intents))

In [2]:
def function_for_greetings():
    print("Greetings intent!")

mappings = {'greeting' : function_for_greetings}

medi_bot = MediDocBot('medi_bot.json', intent_methods=mappings ,model_name="test_model")
medi_bot.train_model()
medi_bot.save_model()

done = False

print("Welcome!")

exit_list = ['exit', 'see you later', 'bye', 'quit', 'break', 'goodbye', 'I am leaving', 'have a good day']

while not done:
    message = input("You: ")
    if message.lower() in exit_list:
        print("Chat with you later!")
        done = True
        break
    else:
        #print("What can I do for you?")
        medi_bot.request(message)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/500
9/9 [==============================] - 1s 8ms/step - loss: 2.4006 - accuracy: 0.0682
Epoch 2/500
9/9 [==============================] - 0s 7ms/step - loss: 2.3911 - accuracy: 0.1136
Epoch 3/500
9/9 [==============================] - 0s 6ms/step - loss: 2.3763 - accuracy: 0.1591
Epoch 4/500
9/9 [==============================] - 0s 7ms/step - loss: 2.3590 - accuracy: 0.1818
Epoch 5/500
9/9 [==============================] - 0s 9ms/step - loss: 2.3560 - accuracy: 0.2273
Epoch 6/500
9/9 [==============================] - 0s 9ms/step - loss: 2.3415 - accuracy: 0.2273
Epoch 7/500
9/9 [==============================] - 0s 8ms/step - loss: 2.3195 - accuracy: 0.3409
Epoch 8/500
9/9 [==============================] - 0s 9ms/step - loss: 2.3126 - accuracy: 0.3182
Epoch 9/500
9/9 [==============================] - 0s 8ms/step - loss: 2.2909 - accuracy: 0.3864
Epoch 10/500
9/9 [==============================] - 0s 9ms/step - loss: 2.2865 - accuracy: 0.3636
Epoch 11/500
9/9 [===========